In [1]:
import numpy as np
import re
import pulp

In [2]:
patterns=[]
cables=[]
length=[]
joltages=[]
with open('test_10.txt') as f:
    for line in f:
        m=re.match(r"\[([.#]+)\] (\([0-9, ()]+\)) \{([0-9,]+)\}",line) 
        patterns.append(m.group(1).replace('.','0').replace("#","1"))
        cables.append(m.group(2))
        joltages.append(np.array(m.group(3).split(','),dtype=int))
#print(patterns,cables)
links=[]
for cable in cables:
    flips=[]
    for link in cable.split(' '):
        flips.append(list(map(int,link.strip("()").split(','))))
    links.append(flips)
    length.append(len(flips))
#    links.append([cable.split(' ')])
print(max(length))
print(joltages)

6
[array([3, 5, 4, 7]), array([ 7,  5, 12,  7,  2]), array([10, 11, 11,  5, 10,  5])]


In [3]:
binary=[]
bitcount=[]
for i in range(max(length)):
    binarystrings=[]
    bits=[]
    for k in range(1,2**(i+1)):
        binarystrings.append(format(k,'0'+str(i+1)+'b'))
        bits.append(format(k,'0'+str(i+1)+'b').count('1'))
    binarystrings=np.array(binarystrings)
    bits=np.array(bits)
    idx = np.argsort(bits)

    binary.append((binarystrings)[idx])
    bitcount.append((bits)[idx])


In [4]:
totalpress=0
for pattern,flips in zip(patterns,links):
    recipe=binary[len(flips)-1]
    bitcounts=bitcount[len(flips)-1]
    #print(pattern,flips,recipe,bitcounts)
    target=np.array(list(pattern),dtype=int)
    for sequence,nums in zip(recipe,bitcounts):
        #print('trying', sequence)
        state=np.zeros(len(pattern),dtype=int)
        for bno,button in zip(range(len(sequence)),list(sequence)):
            if button=='0':
            #    print('Skip',bno)
                continue
            #print (flips[bno],state)
            state[flips[bno]] = (state[flips[bno]]+1) % 2
            #print(state)
        if all(state==target):
            print("found",sequence,nums)
            totalpress+=nums
            break
print(totalpress)

found 010100 2
found 00111 3
found 0110 2
7


In [ ]:
totalpress=0
for joltage,flips in zip(joltages,links):
    print(joltage,flips)
    lprob = pulp.LpProblem("Presses",pulp.LpMinimize)
    k=len(flips)
    for button in range(k):
        

[3 5 4 7] [[3], [1, 3], [2], [2, 3], [0, 2], [0, 1]]
[ 7  5 12  7  2] [[0, 2, 3, 4], [2, 3], [0, 4], [0, 1, 2], [1, 2, 3, 4]]
[10 11 11  5 10  5] [[0, 1, 2, 3, 4], [0, 3, 4], [0, 1, 2, 4, 5], [1, 2]]
